# Tutorial 4: MBHBs & MCMC

张铭昊 北京化工大学 zmh780674484@163.com

In [ ]:
教程4：大质量黑洞双星系统（MBHBs）与马尔科夫链蒙特卡洛方法（MCMC）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful
from lisatools.utils.constants import *

In the fourth tutorial we will examine MBHB waveforms. We will examine how to generate waveforms, add the LISA response, calculate the Likelihood, and then we will run an MCMC with MBHBs. 

In [ ]:
  在第四个教程中我们将检测MBHB系统的引力波波形。我们将研究波形如何产生，LISA如何响应，如何计算似然值。最后会运行关于MBHB系统的MCMC程序。

## Task 1: Generate IMRPhenomHM waveforms

We will start by generating IMRPhenomHM waveforms for MBHBs. Pick reasonable parameters, build a waveform and plot it against the LISA Sensitivity Curve (`LISASens`) in the characteristic strain representation. **Hint**: set `f_ref = 0.0`. You can access the information after waveform generation as attributes on the class. This may be updated

Useful documentation:
* [PhenomHMAmpPhase](https://mikekatz04.github.io/BBHx/html/user/waveforms.html#bbhx.waveforms.phenomhm.PhenomHMAmpPhase)

In [ ]:
任务1：生成IMRPhenomHM波形
  我们将以产生MBHB系统的IMRPhenomHM波形来开始，选择合理的参数，建立一个波形并对照于LISA的灵敏度曲线(LISASens)，以特征应变的形式画出它。
提示：设置f_ref = 0.0 你可以在类中获得波形建立后的信息作为属性。这里可能会被更新

In [ ]:
# imports 
from bbhx.waveforms.phenomhm import PhenomHMAmpPhase
from lisatools.sensitivity import get_sensitivity

In [ ]:
wave_gen = PhenomHMAmpPhase()

m1 = 2e1
m2 = 6e3
chi1 = 0.5
chi2 = 0.9
dist = 15 * 1e9 * PC_SI
phi_refer = 0.5
f_refer = 0.0
t_refer = 1e5
length = 1024

wave_gen(m1, m2, chi1, chi2, dist, phi_ref, f_ref, t_ref, length)

In [ ]:
fn = np.logspace(-5, -1, 10000)
Sn_char_strain = get_sensitivity(fn, sens_fn="LISASens", return_type="char_strain") 


plt.loglog(fn, Sn_char_strain)
for i in range(6):
    char_strain = wave_gen.freqs * wave_gen.amp[0, i]
    plt.loglog(wave_gen.freqs, char_strain)

plt.xlabel("Fre (Hz)")
plt.ylabel("Ch Stra")

## Task 2: Add the response. 

Now repeat the same task as above by adding the response. In `BBHx`, the response is added automatically for you using the main waveform production class: `BBHWaveformFD`. The sensitivity for this should be `A1TDISens`.

Useful Documentation:
* [BBHWaveformFD](https://mikekatz04.github.io/BBHx/html/user/main.html#bbhx.waveformbuild.BBHWaveformFD)

In [ ]:
任务2：添加响应
  现在重复上面的工作来添加响应。在BBHx，响应已经自动地为了你使用BBHWaveformFD类而添加好。灵敏度应为A1TDISens

In [ ]:
# imports
from bbhx.waveformbuild import BBHWaveformFD

In [ ]:
tdi_wave_gen = BBHWaveformFD()
m1 = 2e1
m2 = 6e3
chi1 = 0.5
chi2 = 0.9
dist = 15 * 1e9 * PC_SI
phi_refer = 0.5
f_refer = 0.0
t_refer = 1e5
length = 1024
Tobs = YRSID_SI / 12.  
dt = 10.0  # sec
N = int(Tobs / dt)
Tobs = N * dt
freqs = np.fft.rfftfreq(N, dt)

AET = tdi_wave_gen(
    m1,
    m2, 
    chi1,
    chi2,
    dist, 
    phi_refer,
    f_refer, 
    inc,
    lam,
    beta,
    psi,
    t_refer,
    length=1024, 
    combine=False,  
    direct=False,
    fill=True,
    squeeze=True,
    freqs=freqs
)
fn = np.logspace(-5, -1, 10000)
Sn_char_strain = get_sensitivity(fn, sens_fn="A1TDISens", return_type="char_strain")
plt.loglog(fn, Sn_char_strain)
plt.loglog(freqs, freqs * np.abs(AET[0,0]))

## Task 3: Inject an MBHB and compute its SNR

In [ ]:
任务3：注入大质量黑洞双星系统参数（MBHB）并计算信噪比（SNR）

Now we will combine our knowledge of `lisatools` with `bbhx`. Using the same methods above, setup a full `AnalysisContainer` and inject an MBHB signal. Calculate its SNR. You will probably need to wrap the MBHB waveform generation to select out just the 1st array entry. 

In [ ]:
  现在我们将结合lisatools与bbhx的知识。使用与上文相同的方法，设置一个AnalysisContainer并注入MBHB信号，计算它的信噪比。你可能需要对MBHB
波形发生器进行warp操作来选出仅第一列项目

In [ ]:
# imports
from lisatools.sensitivity import AET1SensitivityMatrix
from lisatools.datacontainer import DataResidualArray
from lisatools.analysiscontainer import AnalysisContainer

In [ ]:
def wrap(*args, **kwargs):
    return tdi_wave_gen(*args, **kwargs)[0]
data = DataResidualArray(AET[0], f_arr=freqs)
sens_mat = AET1SensitivityMatrix(data.f_arr)
analysis = AnalysisContainer(data, sens_mat, signal_gen=wrap)
analysis.snr()

Calculate the Likelihood using `calculate_signal_likelihood`. 

In [ ]:
使用calculate_signal_likelihood计算似然值

In [ ]:
analysis.calculate_signal_likelihood(
    m1 * 1.0001,
    m2, 
    chi1,
    chi2,
    dist, 
    phi_refer,
    f_refer, 
    inc,
    lam,
    beta,
    psi,
    t_refer,
    waveform_kwargs=dict(
        length=1024, 
        combine=False,  
        direct=False,
        fill=True,
        squeeze=True,
        freqs=freqs
    ),
    source_only=True

)

## Task 4: MCMC with MBHBs

Now we will run an MCMC for MBHBs. Use your knowledge from our tutorial on `Eryn` to run an MCMC for MBHBs. Let's run it for over four parameters: `(m1, m2, phi_ref, t_ref)`. You can use an `Eryn` [TransformContainer](https://mikekatz04.github.io/Eryn/html/user/utils.html#transformcontainer) or a wrapper Likelihood to fill in fixed parameters. You can use the `periodic` kwarg for `EnsembleSampler` to run `phi_ref` as a periodic variable wrapping over the edge.

This will not be fast. Run the sampler for a small number of samples and move on to the next step. 

In [ ]:
任务4：关于大质量黑洞双星系统（MBHBs）的马尔科夫蒙特卡洛方法（MCMC）
  现在我们将运行关于MBHB的MCMC程序。应用你之前在教程中学到的关于Eryn的知识来建立一个关于MBHB的MCMC程序。让我们用四个参数来设置它  
(m1, m2, phi_ref, t_ref)。你可以使用Eryn TransformContainer或包装似然来填写固定参数。你可以使用EnsembleSampler的periodic来运行
phi_ref，使其作为一个在边界上环绕的周期变量。
  这个过程不会很快。在小规模样本的情况下运行采样器并跳转到下一步。

In [ ]:
# imports
from eryn.ensemble import EnsembleSampler
from eryn.prior import ProbDistContainer, uniform_dist
from eryn.state import State

In [ ]:
def wrapper_likelihood(x, fixed_parameters, freqs, analysis, **kwargs):
    all_parameters = np.zeros(12)
    mT = x[0]
    q = x[1]
    all_parameters[0] = mT / (1 + q)
    all_parameters[1] = mT * q / (1 + q)
    all_parameters[5] = x[2]
    all_parameters[-1] = x[3]

    all_parameters[np.array([2, 3, 4, 6, 7, 8, 9, 10])] = fixed_parameters

    ll = analysis.calculate_signal_likelihood(
        *all_parameters,
        waveform_kwargs=dict(
            length=1024, 
            combine=False,  # TODO: check this
            direct=False,
            fill=True,
            squeeze=True,
            freqs=freqs
        ),
        source_only=True
        # data_arr_kwargs=dict(f_arr=freqs)
    )
    return ll

## Task 5: Add heterodyning for speed

That would take a long time to complete a sampling run. One technique for speeding up waveforms is called `heterodyning`. Wrap the Heterodyned likelihood to use a subset of parameters. You can also use an Eryn `TransformFunction`. 

Useful Documentation:

* [HeterodynedLikelihood](https://mikekatz04.github.io/BBHx/html/user/like.html#bbhx.likelihood.HeterodynedLikelihood)

In [ ]:
任务5：添加速率的混频
  完成采样器的运行可能花费很长时间。有一种叫做混频的技术可以加速波形产生。包装混频似然值可以达到使用参数子集的目的。你也可以使用Eryn中的TransformFunction

In [ ]:

from bbhx.likelihood import HeterodynedLikelihood
length_f_het = 128

mT = injection_params[0]
q = injection_params[1]

transformed_injection_params = injection_params.copy()
transformed_injection_params[0] = mT / (1 + q)
transformed_injection_params[1] = mT * q / (1 + q)

like_het = HeterodynedLikelihood(
    tdi_wave_gen,
    freqs,
    data[:],
    transformed_injection_params,
    length_f_het,
)

like_het.get_ll(transformed_injection_params[None, :].T)
array([-0.00275878])
def het_wrapper_likelihood(x, fixed_parameters, freqs, het_like):
    all_parameters = np.zeros(12)
    mT = x[0]
    q = x[1]
    all_parameters[0] = mT / (1 + q)
    all_parameters[1] = mT * q / (1 + q)

    # 
    all_parameters[5] = x[2]
    all_parameters[-1] = x[3]

    all_parameters[np.array([2, 3, 4, 6, 7, 8, 9, 10])] = fixed_parameters

    ll = like_het.get_ll(all_parameters)
    return ll
ll_comp = wrapper_likelihood(injection_params[np.array([0, 1, 5, 11])], fixed_parameters, freqs, analysis)
ll_het = het_wrapper_likelihood(injection_params[np.array([0, 1, 5, 11])], fixed_parameters, freqs, like_het)
print(ll_het, ll_comp)
nwalkers = 32
het_sampler = EnsembleSampler(
    nwalkers,
    ndims,
    het_wrapper_likelihood,
    priors,
    args=(fixed_parameters, freqs, like_het),
    branch_names=["mbh"],
    tempering_kwargs=dict(ntemps=ntemps),
    periodic=periodic
)
fig = c.plotter.plot()

Let's check how well the heterodyning method matches the base likelihood by sampling our prior. You will have to update the reference template for each computation since we are drawing from the prior and may be far away from our true point. You can do this with `HeterodynedLikelihood.init_heterdyne_info`. 

In [ ]:
  让我们查看一下使用混频方法通过采集先验分布得到的结果与基似然值匹配的如何。你需要更新每一次计算的参考模板，因为我们抽离了先验分布从而可能
与真实值差距很远。你可以使用HeterodynedLikelihood.init_heterdyne_info 完成这项任务

In [ ]:
num = 10
for params in priors["mbh"].rvs(num):
    update_params = injection_params.copy()
    mT = params[0]
    q = params[1]
    m1 = mT / (1 + q)
    m2 = mT * q / (1 + q)
    update_params[np.array([0, 1, 5, 11])] = np.array([m1, m2, params[2], params[3]])
    like_het.init_heterodyne_info(
        update_params
    )
    ll_comp = wrapper_likelihood(params, fixed_parameters, freqs, analysis)
    ll_het = het_wrapper_likelihood(params, fixed_parameters, freqs, like_het)
    print(ll_comp, ll_het, np.abs(ll_comp - ll_het), np.abs(ll_comp - ll_het) / np.abs(ll_comp))

Setup and run the sampler. Then plot the posteriors using `ChainConsumer` or `corner`.

In [ ]:
  设定好并运行采样器。然后用ChainConsumer或corner绘制出后验分布

In [ ]:

def log_prob(params):
    update_params = injection_params.copy()
    mT = params[0]
    q = params[1]
    m1 = mT / (1 + q)
    m2 = mT * q / (1 + q)
    update_params[np.array([0, 1, 5, 11])] = np.array([m1, m2, params[2], params[3]])
    like_het.init_heterodyne_info(update_params)
    ll_comp = wrapper_likelihood(params, fixed_parameters, freqs, analysis)
    ll_het = het_wrapper_likelihood(params, fixed_parameters, freqs, like_het)
    return ll_comp + ll_het


nwalkers = 100
ndim = 4 
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)

space
p0 = priors["mbh"].rvs(size=nwalkers)


nsteps = 1000  
pos, prob, _ = sampler.run_mcmc(p0, nsteps)


samples = sampler.get_chain(flat=True)
log_prob_samples = sampler.get_log_prob(flat=True)


c = ChainConsumer()
labels = ["mT", "q", "param1", "param2"]
c.add_chain(samples, parameters=labels, name="Posterior")
c.configure(summary=True)
fig = c.plotter.plot()



总结与收获
  这一章依旧涉及MCMC，让我还没有摆脱前一张云里雾里的感觉，但是好歹应用实例更多了一些，这回分析了黑洞双星，这样能让我对这种方法更有感觉。
  令我印象深刻的是后面使用的混频技术，通过混频能将信号转移到更低的频率范围，从而减小信号处理所需的计算量。通过降低信号的频率，可以在更低的采样率下进行处理，从而提高处理速度。理解这些只需要基于基本的奈奎斯特抽样定理，这不由得让我觉得这一思想无比精妙。
  